This notebook just for try to train in dev data before deep dive in training data

this is important:
https://groups.google.com/g/sigarab/c/8e3Os4Q0Aac

# Trying to learn how to do NLP and LLms Predictions
ref :
https://towardsdatascience.com/machine-learning-word-embedding-and-predicting-e603254e4d7b

In [1]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical

docs = ["Data Science is a popular field.", 
         "Data Science is useful in predicting financial performance of economies", 
         "Performance is important in data science"]
tokenizer = Tokenizer()
tokenizer.fit_on_texts(docs)
vocab_size = len(tokenizer.word_index) + 1
model = Sequential()
model.add(Embedding(vocab_size, 3, input_length=1))
model.compile('rmsprop', 'mse')
input_array = tokenizer.texts_to_sequences(["Data"])
output_array = model.predict(input_array)
output_array

1/1 [==============================] - 0s 34ms/step


array([[[ 0.04165288, -0.00589814,  0.03569147]]], dtype=float32)

In [2]:
data = ""
with open("./metamorphosis.txt") as f:
    data = f.read().strip()



tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])
# determine the vocabulary size
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary Size: %d' % vocab_size)
# create line-based sequences
sequences = list()
# print(data.split("\n"))
for encoded in tokenizer.texts_to_sequences(data.split("\n")):
    if len(encoded) > 0:
        print(encoded)
        for i in range(0, len(encoded) - 2):
            sequences.append(encoded[i:i+3])

print('Total Sequences: %d' % len(sequences))
max_length = max(len(seq) for seq in sequences)
print("max_length: %d" % max_length)
sequences = np.array(sequences)
X, y = sequences[:,:-1], to_categorical(sequences[:,-1], num_classes=vocab_size)
print(X.shape)
print(y.shape)

Vocabulary Size: 3247
[1577, 43, 97, 253, 5, 578]
[27, 253, 75, 17, 1, 143, 5, 269, 681, 8, 1, 226, 167, 3]
[190, 68, 1107, 5, 1, 682, 22, 52, 847, 3, 14, 168, 52, 1108]
[848, 31, 239, 270, 9, 240, 9, 105, 34, 849, 143, 9, 138, 1, 148]
[5, 1, 43, 97, 191, 683, 14, 27, 253, 34, 684]
[22, 352, 97, 353, 32, 31, 115, 15, 440, 8, 1, 226, 167]
[31, 192, 35, 2, 685, 1, 331, 5, 1, 579, 86, 31, 115, 440]
[87, 332, 27, 253]
[27, 75, 11, 1578, 43, 97, 253, 1579, 506]
[282, 1109, 1, 139, 1580, 8, 27, 1110]
[1581, 578]
[1582, 1111, 1112]
[1583, 1113, 1114]
[1584, 850, 1585, 1586, 1587, 253, 1588]
[190, 686, 1115, 1589, 1590, 1591]
[1592, 1593]
[393, 5, 1, 43, 97, 253, 578]
[578]
[46, 1111, 1112]
[1594, 46, 1113, 1114]
[74]
[56, 169, 62, 18, 116, 1116, 29, 1595, 1117, 4, 241]
[44, 1596, 8, 6, 131, 54, 11, 851, 1597, 4, 181, 24, 6]
[1598, 93, 49, 3, 32, 4, 507, 6, 94, 11, 76, 4, 33, 84, 6]
[687, 852, 394, 1599, 3, 1600, 46, 1601, 54, 853, 1118]
[1, 1602, 7, 227, 149, 2, 1603, 9, 3, 132, 688, 2, 1604,

In [3]:
# define model
i = tf.keras.layers.Input(shape=(X.shape[1]))
e = tf.keras.layers.Embedding(vocab_size, 10, input_length=max_length)(i)
l = tf.keras.layers.LSTM(10)(e)
d = tf.keras.layers.Dense(vocab_size, activation='softmax')(l)
model = tf.keras.Model(inputs=i, outputs=[d])
print(model.summary())
# compile network
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

tf.keras.utils.plot_model(model)

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 2)]               0         
                                                                 
 embedding_12 (Embedding)    (None, 2, 10)             32470     
                                                                 
 lstm_3 (LSTM)               (None, 10)                840       
                                                                 
 dense_3 (Dense)             (None, 3247)              35717     
                                                                 
Total params: 69027 (269.64 KB)
Trainable params: 69027 (269.64 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


<IPython.core.display.Image object>

In [4]:
model.fit(X, y, epochs=4, verbose=2)

Epoch 1/4
661/661 - 3s - loss: 6.8120 - accuracy: 0.0539 - 3s/epoch - 4ms/step
Epoch 2/4
661/661 - 2s - loss: 6.1685 - accuracy: 0.0544 - 2s/epoch - 3ms/step
Epoch 3/4
661/661 - 2s - loss: 6.1064 - accuracy: 0.0544 - 2s/epoch - 3ms/step
Epoch 4/4
661/661 - 2s - loss: 6.0472 - accuracy: 0.0544 - 2s/epoch - 3ms/step


Vocabulary Size: 15420
Total Sequences: 21131
max_length: 3


In [6]:
text = " there's still so"
text = " ".join(text.split(" ")[:3])
encoded = tokenizer.texts_to_sequences([text])[0]
encoded = np.array([encoded])
next = model.predict(encoded, verbose=0)
for x in next:
    next_word_token = np.argmax(x)
    # map predicted word index to word
    for word, index in tokenizer.word_index.items():
        if index == next_word_token:
            print(word + " ")

the 


In [ ]:
with open('dev.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

In [7]:
import pandas as pd

df = pd.DataFrame(data)

df.info()

"""
This is base frameworks used:
 - Electra (Clark et al., 2020) 
 - BERT (Devlin et al., 2019),


in our dataframe Embedding:
- AraELECTRA (Antoun et al., 2021)    -> electra 
- AraBERTv2 (Antoun et al., 2020)     ->  bertseg
- camelBERT-MSA (Inoue et al., 2021)  -> bertmsa
"""

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3921 entries, 0 to 3920
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   id       3921 non-null   object
 1   word     3921 non-null   object
 2   pos      3636 non-null   object
 3   gloss    3921 non-null   object
 4   electra  3921 non-null   object
 5   bertseg  3921 non-null   object
 6   bertmsa  3921 non-null   object
dtypes: object(7)
memory usage: 214.6+ KB


'\nThis is base frameworks used:\n - Electra (Clark et al., 2020) \n - BERT (Devlin et al., 2019),\n\n\nin our dataframe Embedding:\n- AraELECTRA (Antoun et al., 2021)    -> electra \n- AraBERTv2 (Antoun et al., 2020)     ->  bertseg\n- camelBERT-MSA (Inoue et al., 2021)  -> bertmsa\n'

In [8]:
from transformers import AutoTokenizer, AutoModel
import pandas as pd

# Load pre-trained models and tokenizers
tokenizer_electra = AutoTokenizer.from_pretrained("aubmindlab/bert-base-arabicelectra")
model_electra = AutoModel.from_pretrained("aubmindlab/bert-base-arabicelectra")

# tokenizer_bertseg = AutoTokenizer.from_pretrained("asafaya/bert-base-arabic")
# model_bertseg = AutoModel.from_pretrained("asafaya/bert-base-arabic")

# tokenizer_bertmsa = AutoTokenizer.from_pretrained("CAMeL-Lab/camelbert-base-both")
# model_bertmsa = AutoModel.from_pretrained("CAMeL-Lab/camelbert-base-both")

# # Tokenize the glosses in your DataFrame
# tokenized_glosses = {
#     'electra': [tokenizer_electra(gloss, return_tensors="pt") for gloss in df['gloss']],
#     'bertseg': [tokenizer_bertseg(gloss, return_tensors="pt") for gloss in df['gloss']],
#     'bertmsa': [tokenizer_bertmsa(gloss, return_tensors="pt") for gloss in df['gloss']]
# }

# # Get contextual embeddings for the tokenized glosses
# contextual_embeddings = {
#     'electra': [model_electra(**tokens).last_hidden_state.mean(dim=1).squeeze().tolist() for tokens in tokenized_glosses['electra']],
#     'bertseg': [model_bertseg(**tokens).last_hidden_state.mean(dim=1).squeeze().tolist() for tokens in tokenized_glosses['bertseg']],
#     'bertmsa': [model_bertmsa(**tokens).last_hidden_state.mean(dim=1).squeeze().tolist() for tokens in tokenized_glosses['bertmsa']]
# }

# # Add contextual embeddings to DataFrame
# df['electra_embeddings'] = contextual_embeddings['electra']
# df['bertseg_embeddings'] = contextual_embeddings['bertseg']
# df['bertmsa_embeddings'] = contextual_embeddings['bertmsa']

# Now your DataFrame contains contextual embeddings for each gloss
print(df.head())

---------------------------------------------------------------------------
HTTPError                                 Traceback (most recent call last)
File ~/.local/lib/python3.11/site-packages/huggingface_hub/utils/_errors.py:261, in hf_raise_for_status(response, endpoint_name)
    260 try:
--> 261     response.raise_for_status()
    262 except HTTPError as e:

File ~/.local/lib/python3.11/site-packages/requests/models.py:1021, in Response.raise_for_status(self)
   1020 if http_error_msg:
-> 1021     raise HTTPError(http_error_msg, response=self)

HTTPError: 404 Client Error: Not Found for url: https://huggingface.co/aubmindlab/bert-base-arabicelectra/resolve/main/tokenizer_config.json

The above exception was the direct cause of the following exception:

RepositoryNotFoundError                   Traceback (most recent call last)
File ~/.local/lib/python3.11/site-packages/transformers/utils/hub.py:429, in cached_file(path_or_repo_id, filename, cache_dir, force_download, resume_downlo

In [ ]:
glosses = [i['gloss'] for i in data]
words = [i['word'] for i in data]

In [ ]:
# print(glosses + words)

# Tokenize the glosses and words
tokenizer = Tokenizer()
# tokenizer.fit_on_texts([gloss.split(" ") for gloss in glosses])
tokenizer.fit_on_texts(glosses)
tokenizer.fit_on_texts(words)

# print(tokenizer.word_index)
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary Size: %d' % vocab_size)

# Convert text to sequences
X = tokenizer.texts_to_sequences(glosses)
y = tokenizer.texts_to_sequences(words)

# sequences = list()
# for encoded in tokenizer.texts_to_sequences(glosses):
#     if len(encoded) > 0:
#         for i in range(0, len(encoded) - 2):
#             sequences.append(encoded[i:i+3])


# Pad sequences to ensure uniform length
max_len = max(len(seq) for seq in X)
X_padded = pad_sequences(X, maxlen=max_len)
y_padded = pad_sequences(y, maxlen=max_len)

# Prepare the data for training
# X_train, X_test, y_train, y_test = train_test_split(X_padded, y_padded, test_size=0.2, random_state=42)

# print(X)
# print(y)
# print(X_train)
# print('Total Sequences: %d' % len(sequences))
# max_length = max(len(seq) for seq in sequences)
# print("max_length: %d" % max_length)
# sequences = np.array(sequences)
X, y = X_padded, to_categorical(y_padded, num_classes=vocab_size)

# My stuff

In [ ]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split


from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error


from arabic_reshaper import arabic_reshaper
from bidi.algorithm import get_display

from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
with open('dev.json', 'r', encoding='utf-8') as file:
    dev = json.load(file)

In [9]:
# Print keys to understand the structure
print(dev[0].keys())
print(len(dev))

dict_keys(['id', 'word', 'pos', 'gloss', 'electra', 'bertseg', 'bertmsa'])
3921


In [10]:
for i in dev[:10]:
    print(i['id'], i['word'], i["pos"], i["gloss"])

ar.962714 أَكْمَدَ V غَمَّ وَأَمْرَضَ القَلْبَ
ar.994971 ب None حرف جرّ زائد في التوكيد بالنفس والعين .
ar.989034 ذَكَا V ذَكَا الشَّاةَ ونحوَها: ذَبَحَها
ar.994539 وَرَع V وَرَعَ فلانٌ: جَبُنَ.
ar.992756 قَلَص V قَلَصَ الظِّلُّ عنِّي: انْقَبَض ونَقَصَ.
ar.993508 أَمْرَضَ V أَمْرَضَ فلانٌ: قَارَبَ الصَّوابَ في الرَّأْي وإِن لم يُصِبْ كلَّ الصَّواب.
ar.977791 عليق N : ما يُقدَّم للدَّابَّة من طعام "عليقٌ من شعير/ حشيش- عليقٌ يابس".
ar.977267 تطوع V تبرَّع به، وزاد على ما يجب عليه
ar.968835 قَصْر اَلرِّئَاسَةِ None مقرّ إقامة رئيس الدولة.
ar.978831 مقدم N من يلْتَمس شيئًا .


In [ ]:
data =  dev 

In [ ]:
# Extract glosses and words from the data
glosses = [get_display(arabic_reshaper.reshape(i['gloss'])) for i in data]
words = [i['word'] for i in data]

# Tokenize the glosses and words
tokenizer = Tokenizer()
tokenizer.fit_on_texts(glosses + words)

# Convert text to sequences
X = tokenizer.texts_to_sequences(glosses)
y = tokenizer.texts_to_sequences(words)

# Pad sequences to ensure uniform length
max_len = max(len(seq) for seq in X)
X_padded = pad_sequences(X, maxlen=max_len)
y_padded = pad_sequences(y, maxlen=max_len)

# Prepare the data for training
X_train, X_test, y_train, y_test = train_test_split(X_padded, y_padded, test_size=0.2, random_state=42)

In [ ]:
# Define the model architecture
embedding_dim = 50
model = Sequential([
    Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=embedding_dim, input_length=max_len),
    LSTM(units=64, return_sequences=True),
    Dense(units=len(tokenizer.word_index) + 1, activation='softmax')
])

In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [11]:
model.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_10 (Embedding)    (None, 82, 50)            771000    
                                                                 
 lstm_1 (LSTM)               (None, 82, 64)            29440     
                                                                 
 dense_1 (Dense)             (None, 82, 15420)         1002300   
                                                                 
Total params: 1802740 (6.88 MB)
Trainable params: 1802740 (6.88 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [12]:
# Train the model
model.fit(X_train, y_train, epochs=1, batch_size=32, validation_data=(X_test, y_test))

98/98 [==============================] - 23s 226ms/step - loss: 4.3085 - accuracy: 0.9767 - val_loss: 0.3763 - val_accuracy: 0.9872


---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
Cell In[26], line 1
----> 1 loaded_model.summary()

NameError: name 'loaded_model' is not defined


In [14]:
# Define the path where you want to save the model
model_path = 'model_ar_dev.h5'

# Save the model to disk
model.save(model_path)
print("Model saved successfully to:", model_path)

Model saved successfully to: model_ar_dev.h5


In [15]:
# Load the saved model from disk
loaded_model = load_model(model_path)
print("Model loaded successfully from:", model_path)

Model loaded successfully from: model_ar_dev.h5


In [16]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test[:200], y_test[:200])
print("Test Accuracy:", accuracy)

7/7 [==============================] - 1s 109ms/step - loss: 0.3761 - accuracy: 0.9872
Test Accuracy: 0.9871951341629028


In [17]:
# Generate predictions
predictions = model.predict(X_test[:50])
print(predictions)

2/2 [==============================] - 0s 38ms/step
[[[5.4068747e-04 6.5227629e-05 6.2781997e-05 ... 6.3212283e-05
   6.2815838e-05 7.0873975e-05]
  [2.3483787e-02 6.4803433e-05 5.7790443e-05 ... 5.8451307e-05
   5.7184090e-05 8.1164057e-05]
  [6.2727392e-01 2.5063608e-05 2.0514615e-05 ... 2.0761026e-05
   1.9836778e-05 3.6874189e-05]
  ...
  [9.1977024e-01 5.4175148e-06 4.2600082e-06 ... 4.3101995e-06
   4.0607279e-06 8.5810489e-06]
  [9.1920269e-01 5.4559541e-06 4.2909755e-06 ... 4.3414525e-06
   4.0903751e-06 8.6394703e-06]
  [9.1596699e-01 5.6744811e-06 4.4668404e-06 ... 4.5184083e-06
   4.2587253e-06 8.9698324e-06]]

 [[5.4068747e-04 6.5227629e-05 6.2781997e-05 ... 6.3212283e-05
   6.2815838e-05 7.0873975e-05]
  [2.3483787e-02 6.4803433e-05 5.7790443e-05 ... 5.8451307e-05
   5.7184090e-05 8.1164057e-05]
  [6.2727392e-01 2.5063608e-05 2.0514615e-05 ... 2.0761026e-05
   1.9836778e-05 3.6874189e-05]
  ...
  [9.1835821e-01 5.5127502e-06 4.3368600e-06 ... 4.3877267e-06
   4.1341937e-06

In [18]:
reverse_word_index = {idx: word for word, idx in tokenizer.word_index.items()}
print(reverse_word_index)

{1: 'ﻦﻣ', 2: 'ﻲﻓ', 3: 'ﻭﺃ', 4: 'ﻰﻠﻋ', 5: 'ﻞﻌﻔﻟﺍ', 6: 'ﻰﻟﺇ', 7: 'ﺎﻣ', 8: 'ﻮﻬﻓ', 9: 'ﻥﻼﻓ', 10: 'ﺀﻲﺸﻟﺍ', 11: 'ﻞﻌﻓ', 12: 'ﻪﺑ', 13: 'ﻝﻮﻌﻔﻤﻟﺍﻭ', 14: 'ﻦﻋ', 15: 'ﺚﻧﺆﻣ', 16: 'ﻪﻴﻠﻋ', 17: 'ﺭﺪﺼﻣ', 18: 'ﻊﻤﺟ', 19: 'ﻉﺭﺎﻀﻣ', 20: 'ﺮﻣﻷﺍ', 21: 'ﻪﻟ', 22: 'ﻪﻴﻓ', 23: 'ﺎﻧﻼﻓ', 24: 'ﻞﻌﻔﻟ', 25: 'ﻢﺳﺍ', 26: 'ﻩﻮﺤﻧﻭ', 27: 'ﻲﺘﻟﺍ', 28: 'ﻞﻛ', 29: 'ﻢﺳﻼﻟ', 30: 'ﻻ', 31: 'ﺎﻬﺑ', 32: 'ﺔﻔﺻ', 33: 'ﺏﻮﺴﻨﻣ', 34: 'ﻪﻴﻟﺇ', 35: 'ﻝﻮﻬﺠﻤﻟﺍ', 36: 'ﻲﻨﺒﻣ', 37: 'ﻪﻨﻣ', 38: 'ﻲﺿﺎﻤﻟﺍ', 39: 'ﺎﻬﻴﻓ', 40: 'ﻝﻮﻬﺠﻤﻠﻟ', 41: 'ﺮﻴﻏ', 42: 'ﻱﺬﻟﺍ', 43: 'ﻪﻨﻋ', 44: 'ﺔﻬﺒﺸﻣ', 45: 'ﷲ', 46: 'ﻮﻫﻭ', 47: 'ﺪﻳﺰﻣ', 48: 'ﻥﺎﻛ', 49: 'ﻦﻴﺑ', 50: 'ﻥﺃ', 51: 'ﺀﻲﺷ', 52: 'ﻥﺎﺴﻧﻹﺍ', 53: 'ﻪﻠﻌﺟ', 54: 'ـﺑ', 55: 'ﻲﻫﻭ', 56: 'ﻥﺎﻜﻣ', 57: 'ﺽﺭﻷﺍ', 58: 'ﺮﻣﺃ', 59: 'ﺪﻨﻋ', 60: 'ﺪﻌﺑ', 61: 'ﻢﻟ', 62: 'ﺎﻬﻴﻠﻋ', 63: 'ﺮﻴﺴﻜﺗ', 64: 'ﻞﺟﺮﻟﺍ', 65: 'ﻪﺴﻔﻧ', 66: 'ﺱﺎﻨﻟﺍ', 67: 'ﺀﺎﻤﻟﺍ', 68: 'ﺍﺬﻛ', 69: 'ﺎﻫﻮﺤﻧﻭ', 70: 'ﻻﻭ', 71: 'ﺔﻟﺁ', 72: 'ﻡﻮﻘﻟﺍ', 73: 'ﻊﻣ', 74: 'ﺾﻌﺑ', 75: 'ﻞﻌﺟ', 76: 'ﻞﺜﻣ', 77: 'ﻉﻮﻧ', 78: 'ﺎﻬﻟ', 79: '°', 80: 'ﺪﺣﺃ', 81: 'ﻥﻭﺩ', 82: 'ﻥﻮﻜﻳ', 83: 'ﺕﺍﺫ', 84: 'ﺕﻮﺒﺜﻟﺍ', 85: 'ﻝﺪﺗ', 86: 'ﺎﻬﻨﻣ', 87: 'ﺮﺜﻛﺃ', 8

In [19]:
text = "أَزْهَق  فلان"
text = " ".join(text.split(" ")[:3])
print(text)
encoded = tokenizer.texts_to_sequences([text])[0]
encoded = np.array([encoded])
print(encoded)
next = model.predict(encoded, verbose=0)
print(next)
for x in next:
    next_word_token = np.argmax(x)
    # map predicted word index to word
    for word, index in tokenizer.word_index.items():
        if index == next_word_token:
            print(word + " ")

أَزْهَق  فلان
[[15374]]
[[[9.2438102e-05 6.4691791e-05 6.4691027e-05 ... 6.4708365e-05
   6.4715074e-05 6.5646069e-05]]]


In [20]:
# Convert integer predictions to text for multiple sequences
def sequences_to_text_batch(predictions, tokenizer):
    texts = []
    reverse_word_index = {idx: word for word, idx in tokenizer.word_index.items()}
    for sequence in predictions:
        word_indices = np.argmax(sequence, axis=-1)
        text = ' '.join(reverse_word_index.get(idx, '') for idx in word_indices if idx != 0)
        texts.append(text)
    return texts

# Convert integer predictions to text for a batch of sequences
predicted_texts = sequences_to_text_batch(predictions, tokenizer)
for i, text in enumerate(predicted_texts):
    print("Predicted text for sequence", i + 1, ":", text)

Predicted text for sequence 1 : 
Predicted text for sequence 2 : 
Predicted text for sequence 3 : 
Predicted text for sequence 4 : 
Predicted text for sequence 5 : 
Predicted text for sequence 6 : 
Predicted text for sequence 7 : 
Predicted text for sequence 8 : 
Predicted text for sequence 9 : 
Predicted text for sequence 10 : 
Predicted text for sequence 11 : 
Predicted text for sequence 12 : 
Predicted text for sequence 13 : 
Predicted text for sequence 14 : 
Predicted text for sequence 15 : 
Predicted text for sequence 16 : 
Predicted text for sequence 17 : 
Predicted text for sequence 18 : 
Predicted text for sequence 19 : 
Predicted text for sequence 20 : 
Predicted text for sequence 21 : 
Predicted text for sequence 22 : 
Predicted text for sequence 23 : 
Predicted text for sequence 24 : 
Predicted text for sequence 25 : 
Predicted text for sequence 26 : 
Predicted text for sequence 27 : 
Predicted text for sequence 28 : 
Predicted text for sequence 29 : 
Predicted text for sequ

In [ ]:
def mean_reciprocal_rank(y_true, y_pred):
    mrr = 0
    for true_seq, pred_seq in zip(y_true, y_pred):
        rank = next((i+1 for i, word in enumerate(pred_seq) if word == true_seq[i]), 0)
        mrr += 1 / rank if rank > 0 else 0
    return mrr / len(y_true)


def precision_at_k(y_true, y_pred, k):
    precision = 0
    for true_seq, pred_seq in zip(y_true, y_pred):
        correct_count = sum(1 for true_word, pred_word in zip(true_seq[:k], pred_seq[:k]) if true_word == pred_word)
        precision += correct_count / k
    return precision / len(y_true)

In [ ]:
# Evaluation

def evalute(y_pred, y_test, limit=10, k=5):
    y_pred = y_pred[:limit]
    y_test = y_test[:limit]
    # Compute Cosine Similarity
    # cosine_sim = cosine_similarity(y_pred, y_test)
    # print("Cosine Similarity")
    
    # Compute MSE
    mse = mean_squared_error(y_pred.flatten(), y_test.flatten())
    print("Mean Squared Error:", mse)
    
    # # Ranking
    # mrr = mean_reciprocal_rank(y_test, y_pred)
    # print("Mean Reciprocal Rank:", mrr)
    
    # # k = 5  # Example value of K
    # precision_at_k_5 = precision_at_k(y_test, y_pred, k)
    # print("Precision at", k, ":", precision_at_k_5)

In [ ]:
evalute(predictions, y_test)